# <center> Классификация жестов
<img src='imgs/gesture_digits.png' width='70%'>

[Ссылка](https://drive.google.com/open?id=1XNKW6Gk2xYbKQ-PYpJgJ8-NVCkr3NV24) на датасет с собранными жестами 

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# импортируем требуемые бибилиотеки
from source.network import NNClassifier, Softmax_layer, Flatten
from source.helpme import show_image, load_gestures, create_loader, calculate_pad, show_aug_grid_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

# импортируем функции аугментации
from albumentations import (
    Compose, 
    RandomBrightnessContrast, 
    ShiftScaleRotate, 
    RandomSnow,
    Cutout
)

[Ссылка](https://github.com/albu/albumentations#pixel-level-transforms) на полный список аугментаций

In [ ]:
X, y = load_gestures(['gesture_dataset'], size=(64, 64))

In [ ]:
# с помощью функции show_image мы можем посмотреть как выглядит определенная картинка
show_image(X[47])

In [ ]:
# разобьем выборку на тренировочную и валидационную

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17, shuffle=True)

print('Размер тренировочной выборки: %d' % len(X_train))
print('Размер тестовой выборки: %d' % len(X_test))

X_test = np.transpose(X_test, (0, 3, 1, 2))

In [ ]:
# определим аугментацию
aug = Compose([
    
    Cutout(num_holes=15, max_h_size=10, max_w_size=10, fill_value=0, always_apply=False, p=0.5),
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.1, rotate_limit=35 , border_mode=0, p=0.5),
    RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.4, p=0.5),

])

In [ ]:
# создадим train loader
train_loader = create_loader(X_train, y_train, trs = aug, shuffle=True, batch_size=32)

In [ ]:
# посмотрим как выглядят картинки с примененной аугментацией
show_aug_grid_classification(train_loader, idx=18, size=5)

# Convolutional neural network

In [ ]:
calculate_pad(input_size=64, kernel_size=3, stride=2, output_size=32)

In [ ]:
# определим архитектуру сети

conv_net = torch.nn.Sequential(torch.nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3, stride=1, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(20),
                               # 64
                               
                               torch.nn.Conv2d(in_channels=20,out_channels=30, kernel_size=3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(30),
                               # 32

                               torch.nn.Conv2d(30, 40, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(40),
                               # 16

                               torch.nn.Conv2d(40, 50, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(50),
                               # 8

                               torch.nn.Conv2d(50, 60, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(60),
                               # 4

                               Flatten(),
                               torch.nn.Linear(960, 10))

In [ ]:
# определим модель с удобным функционалом

model = NNClassifier(conv_net, lr=5e-5, criterion=torch.nn.CrossEntropyLoss())

In [ ]:
# обучим нейросеть

model.fit_loader(train_loader, epochs=50,
          valid_data=(X_test, y_test), log_every_epoch=10)

In [ ]:
# посмотрим на кривые обучения

model.show_history()

In [ ]:
# проверим точность модели

model.evaluate_score(X_test, y_test)

In [ ]:
# посмотрим как работает нейросеть

model.show_predict_grid(X_test, y_test, size = 5)

In [ ]:
# model.save('gesture_classifier.pt')

# Transfer-learning

In [ ]:
from torchvision import models
import torchsummary

In [ ]:
X, y = load_gestures(['gesture_dataset'], size=(224, 224))

# разобьем выборку на тренировочную и валидационную
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17, shuffle=True)

print('Размер тренировочной выборки: %d' % len(X_train))
print('Размер тестовой выборки: %d' % len(X_test))

X_test = np.transpose(X_test, (0, 3, 1, 2))

# создадим train loader
train_loader_224 = create_loader(X_train, y_train, trs = aug, shuffle=True, batch_size=32)

In [ ]:
resnet = models.resnet18(pretrained=True)

In [ ]:
for i, param in enumerate(resnet.parameters()):
    param.requires_grad = False

In [ ]:
res_conv_net = torch.nn.Sequential(*(list(resnet.children())[:-3]),
                               
                               torch.nn.Conv2d(in_channels=256,out_channels=256, kernel_size=3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(256),
                               
                               torch.nn.Conv2d(in_channels=256,out_channels=256, kernel_size=3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(256),
                               
                               Flatten(),
                               
                               
                              torch.nn.Linear(4096, 10),
                              )

In [ ]:
torchsummary.summary(res_conv_net, (3, 224, 224), device='cpu')

In [ ]:
# определим модель с удобным функционалом

res_model = NNClassifier(res_conv_net, lr=5e-5, criterion=torch.nn.CrossEntropyLoss())

In [ ]:
# обучим нейросеть

res_model.fit_loader(train_loader_224, epochs=150,
          valid_data=(X_test, y_test), log_every_epoch=2)

In [ ]:
# посмотрим на кривые обучения

res_model.show_history()

In [ ]:
# проверим точность модели

res_model.evaluate_score(X_test, y_test)

In [ ]:
# посмотрим как работает нейросеть

res_model.show_predict_grid(X_test, y_test, size = 8)

In [ ]:
res_model.save('gesture_classifier.pt')